In [1]:
!pip install transformers
!pip install wget

     |████████████████████████████████| 2.1MB 5.2MB/s 
     |████████████████████████████████| 901kB 38.3MB/s 
     |████████████████████████████████| 3.3MB 39.9MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=2341e2d6106f1967021b9d14c0609b1dddeaa9f9265369408f78d05734e82d8c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
# Datenvorbereitung
import json
import wget

# download and open json file
url = 'https://raw.githubusercontent.com/Johannes96/BERT_fineTune/master/data/final_train_ae_CLS.json'
dataset1 = wget.download(url)

with open(dataset1) as json_file:
    train_ae = json.load(json_file)

url =  'https://raw.githubusercontent.com/Johannes96/BERT_fineTune/master/data/final_dev_ae_CLS.json'
dataset2 = wget.download(url)

with open(dataset2) as json_file:
    val_ae = json.load(json_file)

# Extrahiere Sätze und Labels
train_texts = []
train_tags = []
for i in train_ae:
    train_tags.append(train_ae[i]['label'])
    train_texts.append(train_ae[i]['sentence'])

val_texts = []
val_tags = []
for i in val_ae:
    val_tags.append(val_ae[i]['label'])
    val_texts.append(val_ae[i]['sentence'])

# Vereinige Trainings- und Testdaten
texts = train_texts
tags = train_tags
tags.extend(val_tags)
texts.extend(val_texts)

In [3]:
# Bereinige Datensatz von '\x85'
for i, sent in enumerate(texts):
  for i2, word in enumerate(texts[i]):
    if word == '\x85':
      print('sent:', i, 'word:',i2)
      del texts[i][i2]
      del tags[i][i2]

sent: 3600 word: 5
sent: 3655 word: 5
sent: 3924 word: 5
sent: 6585 word: 5


In [4]:
# Teile in Trainigns- und Testdaten auf
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [5]:
unique_tags = set(tag for doc in train_tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}
print(unique_tags)

{'B', 'I', 'O'}


In [8]:
# Tokenization
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('prajjwal1/bert-small')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for i, (doc_labels, doc_offset) in enumerate(zip(labels, encodings.offset_mapping)):
        # create an empty array of -100
        # print(i)
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())
    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [10]:
import torch

class AEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = AEDataset(train_encodings, train_labels)
val_dataset = AEDataset(val_encodings, val_labels)

In [12]:
# Spezifiziere Anzahl der labels
from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained('prajjwal1/bert-small', num_labels=len(unique_tags))

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from 

In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,1.068800
20,1.008100
30,0.885800
40,0.740900
50,0.595600
60,0.434800
70,0.305300
80,0.273500
90,0.285100
100,0.251400


TrainOutput(global_step=1152, training_loss=0.153007394918758, metrics={'train_runtime': 196.58, 'train_samples_per_second': 5.86, 'total_flos': 342913494420504.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 1412599808, 'init_mem_gpu_alloc_delta': 114420224, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 75001856, 'train_mem_gpu_alloc_delta': 343885824, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 345015808})

In [14]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_loss': 0.11502081155776978,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 149401600,
 'eval_runtime': 3.3374,
 'eval_samples_per_second': 459.635}